In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/12 21:10:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df = spark.read.option("header", "true").option('inferSchema', "true").csv('./data/Case.csv')

In [22]:
df.printSchema()

root
 |--  case_id: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [23]:
df = df.withColumnRenamed(" case_id", "case_id")
df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [24]:
from pyspark.sql.functions import col, desc

In [27]:
#df.select(col("province"), col("city"), col("confirmed")).orderBy(col('confirmed').cast("int").desc()).limit(3).show()

df.select(col("province"), col("city"), col("confirmed")).filter((col('city') != '-') & (col('city') != 'from other city')).orderBy(desc(col("confirmed"))).show()

+-----------------+---------------+---------+
|         province|           city|confirmed|
+-----------------+---------------+---------+
|            Seoul|        Guro-gu|       95|
|Chungcheongnam-do|      Seosan-si|        9|
| Gyeongsangnam-do|       Jinju-si|        9|
|           Sejong|         Sejong|        8|
| Gyeongsangnam-do|   Geochang-gun|        8|
|            Seoul|     Gangnam-gu|        7|
|            Seoul|        Jung-gu|        7|
|            Seoul|      Jongno-gu|        7|
|          Daejeon|         Seo-gu|        7|
| Gyeongsangnam-do|    Changwon-si|        7|
| Gyeongsangnam-do|Changnyeong-gun|        7|
| Gyeongsangbuk-do|    Bonghwa-gun|       68|
|      Gyeonggi-do|    Seongnam-si|       67|
|      Gyeonggi-do|     Bucheon-si|       67|
| Gyeongsangbuk-do|   Gyeongsan-si|       66|
|            Seoul|     Gangnam-gu|        6|
|            Seoul|   Geumcheon-gu|        6|
|            Busan|    Haeundae-gu|        6|
|      Gyeonggi-do|   Uijeongbu-si

In [29]:
df2 = spark.read.option("header", "true").option('inferSchema', "true").csv('./data/PatientInfo.csv')

In [30]:
df2.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)



In [ ]:
df2.select(col('patient_id')).distinct().count()

5164

In [34]:
df2 = df2.dropDuplicates(["patient_id"])

In [41]:
from pyspark.sql.functions import count

#df2.filter(col("infected_by").isNull()).count()
df2.select(count('infected_by').alias('conteo')).show()

+------+
|conteo|
+------+
|  1346|
+------+



In [42]:
df2_contagios = df2.na.drop(subset=['infected_by'])

In [43]:
df2_contagios.count()

1346

In [47]:
df3 = df2_contagios.filter(col("infected_by").isNull()).filter(col("sex") == 'female')


In [51]:
df2.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

In [52]:
#disminuyo a 2 
df2.coalesce(2).write.partitionBy('province').mode('overwrite').parquet('./data/salida')